# Dependency Algorithm

In [1]:
import collections

*Context: I was writing a class that would take a list of queries, execute each of them, and return the results. Given the frequency for subsequent queries to rely on tables created from previous queries, I decided to create a dependency algorithm that would automatically order the queries to be run in such an order that resolved all dependencies.*

First, here's an example of some dependencies - in this dictionary, the key to the dictionary is the name of the item, and the value is a list of 0 or more other items that this item is dependent on (there are two mistakes in this dictionary of dependencies that we will catch with our checks below).

In [2]:
# Example dictionary of items we want to resolve dependencies for
my_items = {
    'A': ['B', 'C', 'D'],  # -- A is dependent on B, C, D,
    'B': [],  # -- B is dependent on nothing, etc.
    'C': ['D'],
    'D': ['B', 'E'],
    'E': ['A'],
    'F': [],
    'Z': ['A', 'B', 'C', 'D', 'Y']
}

### Checks

Before proceeding, it is important that we conduct two checks on our items:
1. Do all of the dependencies exist? We can't have an item be dependent on another item that does not exist.
2. Are there any circular dependencies? We can't have an item that is dependent on another item that is dependent on that first item!

In [3]:
def dependencies_exist(my_items):
    all_dependencies_exist = True
    for item, dependencies in my_items.items():
        for dependency in dependencies:
            if dependency not in my_items.keys():
                print 'Non-existant dependency: ({0}, {1})'.format(
                    item, dependency)
                all_dependencies_exist = False
    return all_dependencies_exist


dependencies_exist(my_items)

Non-existant dependency: (Z, Y)


False

`my_items` failed the `dependencies_exist` function because Z is dependent on Y, which does not exist. Let's try deleting this dependency and seeing if `my_items` can pass this check.

In [4]:
my_items['Z'] = ['A', 'B', 'C', 'D']
dependencies_exist(my_items)

True

All good! Now, let's check for circular dependencies. To do this, we loop over the items in `my_dicts`: for each item, we loop over its dependencies and find the dependencies for the dependencies, and so forth, until we no longer find any new dependencies. Cases where a dependency for an item is that item are marked as circular.

In [5]:
def list_dependencies(my_items, item):
    """Find all dependencies for [item] within [my_items].
    """
    dependencies, new_dependencies_count = list(my_items[item]), -1
    while new_dependencies_count != 0:
        new_dependencies_count = 0
        for item in dependencies:
            new_dependencies = [new_item for new_item in my_items[item] 
                                if new_item not in dependencies]
            dependencies += new_dependencies
            new_dependencies_count += len(new_dependencies)
    return dependencies


def no_circular_dependencies(my_items):
    """Check for any circular dependencies in [my_items].
    """
    not_circular = True
    for item in my_items.keys():
        dependencies = list_dependencies(my_items, item)
        if item in dependencies:
            not_circular = False
            print 'Circular dependency for', item
    return not_circular


no_circular_dependencies(my_items)

Circular dependency for A
Circular dependency for C
Circular dependency for E
Circular dependency for D


False

There's a circular dependency between items A and E - notice that A is dependent on D, which id dependent on E, which is dependent on A. One way to get rid of this would be to get rid of E's dependency on A. Let's try this and re-run the check.

In [6]:
my_items['E'] = ['F']
no_circular_dependencies(my_items)

True

### Ordering

Now, we need to order the dependencies such that they resolve correctly if they are run in order. To do this, we create an `order_dependencies` function that accepts as input `my_items`, and returns a list of keys from `my_items` in order. 

The algorithm works like so:
- Initialize the items (keys of `my_items`) in any order
- Set an index to 0
- While the index is not equal to the length of the items minus one (to account for zero-indexing):
- @ Loop over the dependencies for the item in the items list indexed by *index*
- @ If a dependency is to the right of (comes after) the item in the list of items, move the item to the end of the list and re-start the process
- @ If no dependencies are to the right of the item, increment the index by 1
- Once all items are to the right of all of their respective dependencies, the list is ordered

In [7]:
def order_dependencies(my_items):
    """Order the dependencies in [my_items], returning a list of keys from
    [my_items] in order such that all dependencies resolve.
    """
    items, index = my_items.keys(), 0
    while (len(items) - 1) != index:
        item = items[index]
        current_dependencies = list_dependencies(my_items, item)
        no_order_change = True
        for dependency in current_dependencies:
            if items.index(dependency) - items.index(item) > 0:
                items += [items.pop(index)]
                no_order_change = False
                break
        if no_order_change:
            index += 1
    return items


order_dependencies(my_items)

['B', 'F', 'E', 'D', 'C', 'A', 'Z']

In terms of algorithmic complexity, we are looking at Ω(n) as our lower bound - if the list is already in order, we still have to loop over every item in that list to ensure that all dependencies are to the left of each item.

### Creating a `dependencies` class

Finally, let's organize all of the above functions into a handy class for ordering dependencies.

In [8]:
class Dependencies(object):
    """Orders a dictionary of items to their dependencies such that all 
    dependencies resolve. Initialize the class with the [items] object:

    {
        'item1': ['dependency1', 'dependency2', ...],
        'item2': ['dependency1', 'dependency2', ...],
        ...
    }

    Alternatively, the class can be intiailized with no items, and items can 
    be added with the add_item method.

    The order method returns an OrderedDict of self.items in an order that 
    resolves all dependencies, and the order_list method does the same but 
    simply returns a list of item names in order.

    This class also ensures that all dependencies exist, and no circular
    dependencies exist. These checks happen when calling the order or order_list
    methods.
    """
    
    def __init__(self, items = {}):
        """Perform dependency existance and no circular dependencies checks
        when initializing a Dependencies object.
        """
        assert isinstance(items, dict), '[items] must be a dict'
        assert self.dependencies_exist(items), \
            'One or more dependencies do not exist in [items]'
        assert self.no_circular_dependencies(items), \
            'One or more circular dependencies exist in [dependencies]'
        self.items = items.copy()
        
        
    @staticmethod
    def dependencies_exist(my_items):
        """Check if the initial dependencies for the items in [my_items] exist 
        in [my_items].
        """
        all_dependencies_exist = True
        for item, dependencies in my_items.items():
            for dependency in dependencies:
                if dependency not in my_items.keys():
                    print 'Non-existant dependency: ({0}, {1})'.format(
                        item, dependency)
                    all_dependencies_exist = False
        return all_dependencies_exist
    
    
    @staticmethod
    def list_dependencies(my_items, item):
        """Find all dependencies for [item] within [my_items].
        """
        dependencies, new_dependencies_count = list(my_items[item]), -1
        while new_dependencies_count != 0:
            new_dependencies_count = 0
            for item in dependencies:
                new_dependencies = [new_item for new_item in my_items[item] 
                                    if new_item not in dependencies]
                dependencies += new_dependencies
                new_dependencies_count += len(new_dependencies)
        return dependencies

    
    def no_circular_dependencies(self, my_items):
        """Check for any circular dependencies in [my_items].
        """
        not_circular = True
        for item in my_items.keys():
            dependencies = self.list_dependencies(my_items, item)
            if item in dependencies:
                not_circular = False
                print 'Circular dependency for', item
        return not_circular
        
        
    def add_item(self, item, dependencies = []):
        """Add a new item to self.dependencies, along with optional dependencies
        for that item.
        """
        if self.items.has_key(item):
            raise Exception('{0} already an item!'.format(str(item)))
        else:
            self.items[item] = dependencies
            
            
    def remove_item(self, item):
        """Add a new item to self.dependencies, along with optional dependencies
        for that item.
        """
        if not self.items.has_key(item):
            raise Exception('{0} does not exist!'.format(str(item)))
        else:
            del self.items[item]
            
            
    def modify_item(self, item, dependencies = []):
        """Modify the dependencies for an item.
        """
        if not self.items.has_key(item):
            raise Exception('{0} does not exist!'.format(str(item)))
        else:
            self.items[item] = dependencies
            
            
    def order_dependencies(self, my_items):
        """Order the dependencies in [my_items], returning a list of keys from
        [my_items] in order such that all dependencies resolve.
        """
        items, index = my_items.keys(), 0
        while (len(items) - 1) != index:
            item = items[index]
            current_dependencies = self.list_dependencies(my_items, item)
            no_order_change = True
            for dependency in current_dependencies:
                if items.index(dependency) - items.index(item) > 0:
                    items += [items.pop(index)]
                    no_order_change = False
                    break
            if no_order_change:
                index += 1
        return items
    
    
    def order(self):
        """Return self.items as an OrderedDict in an order that resolves
        all dependencies.
        """
        self.__init__(self.items)
        if self.items:
            ordered_items = self.order_dependencies(self.items)
            return collections.OrderedDict(
                [(item, self.items[item]) for item in ordered_items])
        else:
            return {}
        
    
    def order_list(self):
        """Returns the keys of self.items (the item names) in an order 
        that resolves all dependencies.
        """
        self.__init__(self.items)
        if self.items:
            return self.order_dependencies(self.items)
        else:
            return []
    
    
    def __str__(self):
        return str(self.items)
    
    
    __repr__ = __str__


Let's test out the class on `my_items`:

In [9]:
Dependencies(my_items).order()

OrderedDict([('B', []),
             ('F', []),
             ('E', ['F']),
             ('D', ['B', 'E']),
             ('C', ['D']),
             ('A', ['B', 'C', 'D']),
             ('Z', ['A', 'B', 'C', 'D'])])

In [10]:
Dependencies(my_items).order_list() == order_dependencies(my_items)

True

We can also manually add, modify, or remove items from a `Dependencies` object. When adding/deleting/modifying the assertions from `__init__` are not run.

In [11]:
items = Dependencies()
items.add_item('A', ['B', 'C', 'D'])
items.add_item('B')
items.add_item('C', ['D'])
items.add_item('D', ['B', 'E'])
items.add_item('E', ['A'])
items.add_item('F')
items.add_item('Z', ['A', 'B', 'C', 'D', 'Y'])
items.add_item('AA', [])
items

{'A': ['B', 'C', 'D'], 'AA': [], 'C': ['D'], 'B': [], 'E': ['A'], 'D': ['B', 'E'], 'F': [], 'Z': ['A', 'B', 'C', 'D', 'Y']}

Only when we call the `order` or `order_list` methods is `__init__` called and the assertions run.

In [12]:
items.order()

Non-existant dependency: (Z, Y)


AssertionError: One or more dependencies do not exist in [items]

Like before, we need to correct for dependencies that do not exist.

In [13]:
items.modify_item('Z', ['A', 'B', 'C', 'D'])
items.order()

Circular dependency for A
Circular dependency for C
Circular dependency for E
Circular dependency for D


AssertionError: One or more circular dependencies exist in [dependencies]

And, again, we need to fix the circular dependency between A and E.

In [14]:
items.modify_item('E', ['F'])
items.remove_item('AA')
items.order()

OrderedDict([('B', []),
             ('F', []),
             ('E', ['F']),
             ('D', ['B', 'E']),
             ('C', ['D']),
             ('A', ['B', 'C', 'D']),
             ('Z', ['A', 'B', 'C', 'D'])])

All set!